In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate,tune
from models.Interactors import FIM_Interactor, KNRM_Interactor
from models.SFI import SFI_gating, SFI_gating_MultiView
from configs.ManualConfig import hparams

In [2]:
name='sfi'
hparams['k'] = 30
hparams['select'] = 'gating'
hparams['onehot'] = True
hparams['device'] = 'cuda:0'
# hparams['threshold'] = 0.5

In [3]:
# hparams['validate'] = True
vocab, loaders = prepare(hparams, pin_memory=False)

[2021-04-20 12:29:28,116] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 30, 'vert_num': 18, 'subvert_num': 293, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'filter_num': 150, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:0', 'attrs': ['title'], 'k': 30, 'select': 'gating', 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True, 'onehot': True}
[2021-04-20 12:29:28,118] INFO (root) preparing dataset...
[2021-04-20 12:29:32,025] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
record = next(iter(loaders[0]))

In [6]:
from models.Encoders.FIM import FIM_Encoder

encoder = FIM_Encoder(hparams, vocab)
# interactor = KNRM_Interactor()

hparams['name'] = '-'.join([name,encoder.name,hparams['select']])

# sfi = SFI_gating_MultiView(hparams, encoder, interactor).to(hparams['device'])
sfi = SFI_gating(hparams, encoder).to(hparams['device'])

# sfi.load_state_dict(torch.load('data/model_params/sfi-fim-fim-gating/large_epoch2_step16916_[hs=50,topk=50,contra=0,attrs=title].model', map_location=hparams['device'])['model'])

In [7]:
sfi(record)

tensor([[-1.7278, -2.0676, -1.4892, -1.6390, -1.2870],
        [-1.5752, -1.9683, -1.6511, -1.4407, -1.4928],
        [-1.7922, -1.5795, -1.6489, -1.6739, -1.3962],
        [-2.1476, -1.5904, -1.5718, -1.7152, -1.2317],
        [-1.6885, -1.3825, -1.9636, -1.4950, -1.6112],
        [-1.8921, -1.3670, -1.5843, -1.6907, -1.5852],
        [-2.0071, -1.8912, -1.3797, -1.5024, -1.4251],
        [-0.8704, -1.7389, -2.0998, -1.7520, -2.2110],
        [-1.8499, -1.2893, -1.7509, -1.7916, -1.4829],
        [-1.0231, -1.9655, -1.3124, -2.0735, -2.2490]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)

In [10]:
sfi = tune(sfi, hparams, loaders)

[2021-04-20 12:32:07,988] INFO (root) training...
epoch 1 , step 290 , loss: 1.5464: 100%|█████████▉| 294/295 [00:34<00:00,  8.46it/s][2021-04-20 12:32:42,852] INFO (root) evaluating...

  0%|          | 0/1812 [00:00<?, ?it/s]


100%|██████████| 1812/1812 [01:07<00:00, 26.91it/s]
[2021-04-20 12:33:50,725] INFO (root) current result is {'auc': 0.5496, 'mean_mrr': 0.2489, 'ndcg@5': 0.2582, 'ndcg@10': 0.327, 'epoch': 1, 'step': 294}
[2021-04-20 12:33:50,727] INFO (root) best result till now is {'auc': 0.5496, 'mean_mrr': 0.2489, 'ndcg@5': 0.2582, 'ndcg@10': 0.327, 'epoch': 1, 'step': 294}
epoch 1 , step 290 , loss: 1.5464: 100%|█████████▉| 294/295 [01:42<00:00,  2.86it/s]


TypeError: object of type 'NoneType' has no len()

In [ ]:
evaluate(sfi, hparams, loaders[1], loading=True)